In [1]:

#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Helper libraries
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import math

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import model_from_json


2023-03-29 19:27:16.184004: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 19:27:16.948231: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-29 19:27:19.162114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/
2023-03-29 19:27:19.162227: W t

Num GPUs Available:  1


## Data Splitting ##

In [2]:
# df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
# df['filename'] = df["image_id"].astype(str) + ".png"
# df = df.sample(frac=1).reset_index(drop=True)
# df.to_csv("training_norm_shuffle.csv", encoding='utf-8', index=False)

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm_shuffle.csv')
print(df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0           414  0.8125    1.0    414.png
1         17532  0.6875    1.0  17532.png
2         16480  0.3125    0.0  16480.png
3          6450  0.6875    0.0   6450.png
4         12314  0.6250    0.0  12314.png
...         ...     ...    ...        ...
14821      9844  0.6875    1.0   9844.png
14822     12367  0.5000    1.0  12367.png
14823       343  0.9375    1.0    343.png
14824      3838  0.6875    0.0   3838.png
14825     13301  0.5000    1.0  13301.png

[14826 rows x 4 columns]


## Data Preprocessing ##

In [3]:

batch_size = 32
img_size = [240, 320]
shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift
)

speed_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

angle_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

speed_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

speed_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="speed",
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col="angle",
    target_size=img_size,
    shuffle=False,
    class_mode='other')



Found 8895 validated image filenames.
Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.
Found 2966 validated image filenames.


In [4]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)
submission_datagen = ImageDataGenerator(rescale = 1./255)

submission_images = submission_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.


## Transfer Learning ##

In [20]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-2]:
    layer.trainable = False

## Model Creation and Evaluation ##

In [21]:
# Speed CNN base
speed_model = models.Sequential()

speed_model.add(transfer)
speed_model.build()
        
speed_model.add(layers.Conv2D(1280, (3, 3), strides=(2,2), activation='relu'))
speed_model.add(layers.BatchNormalization())

speed_model.add(layers.Flatten()),

speed_model.add(layers.Dense(128, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

speed_model.add(layers.Dense(64, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

# Output layer
speed_model.add(layers.Flatten())
speed_model.add(layers.Dense(1,  activation='sigmoid'))

speed_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 4, 1280)        14746880  
                                                                 
 batch_normalization_5 (Batc  (None, 3, 4, 1280)       5120      
 hNormalization)                                                 
                                                                 
 flatten_4 (Flatten)         (None, 15360)             0         
                                                                 
 dense_5 (Dense)             (None, 128)               1966208   
                                                                 
 batch_normalization_6 (Batc  (None, 128)             

In [22]:
# Angle CNN base
angle_model = models.Sequential()

angle_model.add(transfer)
angle_model.build()
        
angle_model.add(layers.Conv2D(128, (3, 3), strides=(2,2), activation='relu'))
angle_model.add(layers.BatchNormalization())

angle_model.add(layers.Flatten()),

# model_1.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.BatchNormalization())
angle_model.add(layers.Dropout(0.2))

# Output layer
angle_model.add(layers.Flatten())
angle_model.add(layers.Dense(1,  activation='relu'))

angle_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 4, 128)         1474688   
                                                                 
 batch_normalization_8 (Batc  (None, 3, 4, 128)        512       
 hNormalization)                                                 
                                                                 
 flatten_6 (Flatten)         (None, 1536)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               196736    
                                                                 
 batch_normalization_9 (Batc  (None, 128)             

In [23]:
#speed_model.build()
#angle_model.build()

In [24]:
speed_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error'
)

angle_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

In [25]:
history = speed_model.fit(
    speed_train_generator,
    batch_size=batch_size,
    validation_data=speed_val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100
278/278 [==============================] - 112s 385ms/step - loss: 0.0525 - val_loss: 0.0221
Epoch 2/100
278/278 [==============================] - 105s 377ms/step - loss: 0.0231 - val_loss: 0.0204
Epoch 3/100
278/278 [==============================] - 97s 349ms/step - loss: 0.0193 - val_loss: 0.0182
Epoch 4/100
278/278 [==============================] - 98s 353ms/step - loss: 0.0164 - val_loss: 0.0159
Epoch 5/100
278/278 [==============================] - 98s 354ms/step - loss: 0.0134 - val_loss: 0.0149
Epoch 6/100
278/278 [==============================] - 99s 357ms/step - loss: 0.0122 - val_loss: 0.0166
Epoch 7/100
278/278 [==============================] - 100s 361ms/step - loss: 0.0109 - val_loss: 0.0142
Epoch 8/100
278/278 [==============================] - 100s 361ms/step - loss: 0.0112 - val_loss: 0.0145
Epoch 9/100
278/278 [==============================] - 100s 361ms/step - loss: 0.0105 - val_loss: 0.0128
Epoch 10/100
278/278 [==============================] - 100

In [26]:
test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)

93/93 [==============================] - 9s 98ms/step - loss: 0.0157


In [27]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100
278/278 [==============================] - 114s 394ms/step - loss: 0.3833 - val_loss: 0.2791
Epoch 2/100
278/278 [==============================] - 107s 384ms/step - loss: 0.2030 - val_loss: 0.1198
Epoch 3/100
278/278 [==============================] - 106s 381ms/step - loss: 0.1127 - val_loss: 0.0751
Epoch 4/100
278/278 [==============================] - 104s 374ms/step - loss: 0.0677 - val_loss: 0.0261
Epoch 5/100
278/278 [==============================] - 94s 337ms/step - loss: 0.0374 - val_loss: 0.0211
Epoch 6/100
278/278 [==============================] - 98s 353ms/step - loss: 0.0209 - val_loss: 0.0109
Epoch 7/100
278/278 [==============================] - 107s 383ms/step - loss: 0.0127 - val_loss: 0.0080
Epoch 8/100
278/278 [==============================] - 107s 383ms/step - loss: 0.0104 - val_loss: 0.0075
Epoch 9/100
278/278 [==============================] - 108s 388ms/step - loss: 0.0098 - val_loss: 0.0075
Epoch 10/100
278/278 [==============================] - 1

In [28]:
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

93/93 [==============================] - 9s 95ms/step - loss: 0.0064


In [29]:
speed_model.save("speed-model.h5")
angle_model.save("angle-model.h5")

## Model Prediction ##

In [31]:
angle_prediction = angle_model.predict(submission_images)
speed_prediction = speed_model.predict(submission_images)

speed_prediction = speed_prediction.flatten()
angle_prediction = angle_prediction.flatten()

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('Submission.csv', index_label="image_id")


32/32 [==============================] - 3s 79ms/step


In [ ]:
diff = abs(x_evaluate["angle"] - angle_prediction)
results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":angle_prediction, "Difference":diff})
#results

In [32]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":speed_prediction})
#results

ValueError: array length 1020 does not match index length 2966

## Fine-tuning ##

In [33]:
speed_model = load_model("speed-model.h5")
angle_model = load_model("angle-model.h5")


angle_transfer = angle_model.layers[0]
angle_transfer.trainable = True

for layer in angle_transfer.layers[:-25]:
    layer.trainable = False
        
angle_model.summary()


speed_transfer = speed_model.layers[0]
speed_transfer.trainable = True

for layer in speed_transfer.layers[:-25]:
    layer.trainable = False
    
speed_model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 4, 128)         1474688   
                                                                 
 batch_normalization_8 (Batc  (None, 3, 4, 128)        512       
 hNormalization)                                                 
                                                                 
 flatten_6 (Flatten)         (None, 1536)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               196736    
                                                                 
 batch_normalization_9 (Batc  (None, 128)             

In [34]:

speed_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001),
    loss='mean_squared_error'
)

angle_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0005,
    patience=30,
    verbose=1,
    mode='min',
    restore_best_weights=True
)


In [35]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=300
)

Epoch 1/300
278/278 [==============================] - 114s 386ms/step - loss: 0.0124 - val_loss: 0.0096
Epoch 2/300
278/278 [==============================] - 115s 415ms/step - loss: 0.0097 - val_loss: 0.0219
Epoch 3/300
278/278 [==============================] - 118s 424ms/step - loss: 0.0094 - val_loss: 0.0159
Epoch 4/300
278/278 [==============================] - 119s 429ms/step - loss: 0.0091 - val_loss: 0.0092
Epoch 5/300
278/278 [==============================] - 118s 426ms/step - loss: 0.0088 - val_loss: 0.0073
Epoch 6/300
278/278 [==============================] - 120s 431ms/step - loss: 0.0085 - val_loss: 0.0070
Epoch 7/300
278/278 [==============================] - 120s 433ms/step - loss: 0.0084 - val_loss: 0.0069
Epoch 8/300
278/278 [==============================] - 119s 428ms/step - loss: 0.0082 - val_loss: 0.0068
Epoch 9/300
278/278 [==============================] - 120s 432ms/step - loss: 0.0080 - val_loss: 0.0066
Epoch 10/300
278/278 [==============================] -

In [36]:
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

93/93 [==============================] - 9s 95ms/step - loss: 0.0058


In [37]:
history = speed_model.fit(
    speed_train_generator,
    batch_size=batch_size,
    validation_data=speed_val_generator,
    callbacks=[es],
    epochs=300
)

Epoch 1/300
278/278 [==============================] - 118s 403ms/step - loss: 0.0131 - val_loss: 0.0104
Epoch 2/300
278/278 [==============================] - 112s 401ms/step - loss: 0.0082 - val_loss: 0.0105
Epoch 3/300
278/278 [==============================] - 111s 398ms/step - loss: 0.0074 - val_loss: 0.0103
Epoch 4/300
278/278 [==============================] - 112s 404ms/step - loss: 0.0066 - val_loss: 0.0114
Epoch 5/300
278/278 [==============================] - 112s 404ms/step - loss: 0.0051 - val_loss: 0.0110
Epoch 6/300
278/278 [==============================] - 113s 405ms/step - loss: 0.0051 - val_loss: 0.0110
Epoch 7/300
278/278 [==============================] - 112s 403ms/step - loss: 0.0041 - val_loss: 0.0115
Epoch 8/300
278/278 [==============================] - 112s 404ms/step - loss: 0.0044 - val_loss: 0.0118
Epoch 9/300
278/278 [==============================] - 113s 405ms/step - loss: 0.0029 - val_loss: 0.0122
Epoch 10/300
278/278 [==============================] -

In [38]:
from sklearn.metrics import classification_report

test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)


93/93 [==============================] - 9s 97ms/step - loss: 0.0167


In [39]:
speed_model.save("speed-model-fine-tuning.h5")
angle_model.save("angle-model-fine-tuning.h5")

In [40]:
angle_prediction = angle_model.predict(submission_images)
speed_prediction = speed_model.predict(submission_images)

speed_prediction = speed_prediction.flatten()
angle_prediction = angle_prediction.flatten()

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('Submission-fine-tuning.csv', index_label="image_id")

32/32 [==============================] - 3s 83ms/step


In [ ]:
model = load_model("model-fine-tuning.hdf5")
plot_model(model, show_layer_names=True, show_shapes=True,show_layer_activations=True)

## Clean Image File ##

In [ ]:
speed_model = load_model("speed-model.h5")
angle_model = load_model("angle-model-fine-tuning.h5")

model = load_model("model.h5")

x_all = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
x_all['filename'] = x_all["image_id"].astype(str) + ".png"

clean_datagen = ImageDataGenerator(rescale = 1./255)

clean_generator = clean_datagen.flow_from_dataframe(
    dataframe=x_all,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

angle_prediction = angle_model.predict(clean_generator)
speed_prediction = speed_model.predict(clean_generator)

# predict = model.predict(clean_generator)

# angle_prediction = predict[:,0]
# speed_prediction = predict[:,1]

diff = abs(x_all["angle"] - angle_prediction)

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

correct = (x_all["speed"] == speed_prediction)

clean_data = {
    "image_id":x_all["image_id"], 
    "Actual Angle":x_all["angle"], 
    "Predicted Angle":angle_prediction, 
    "Angle Difference":diff, 
    "Actual Speed":x_all["speed"], 
    "Predicted Speed":speed_prediction,
    "Correct?":correct
}

df = pd.DataFrame(clean_data)
df.index += 1 
#df.to_csv('clean-results.csv', index_label="image_id")


## Run other models ##

In [18]:
angle_model = load_model("angle-model-fine-tuning.h5")
speed_model = load_model("sakthi-speed2.hdf5")
# best_model = load_model("best-model.h5")

speed_prediction = speed_model.predict(submission_images)
angle_prediction = angle_model.predict(submission_images)
# best_prediction = best_model.predict(submission_images)

# print(speed_prediction)

# speed_prediction = speed_prediction[0]

speed_prediction = speed_prediction.flatten()
angle_prediction = angle_prediction.flatten()


# best_angle = best_prediction[:,0]
# best_speed = best_prediction[:,1]


# best_speed += 0.5
# best_speed = np.floor(best_speed)

# diff = abs(best_angle - angle_prediction)

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

# match = (best_speed == speed_prediction)

# validate = {
#     "Best Angle":best_angle, 
#     "Jarrad Angle":angle_prediction, 
#     "Angle Difference":diff, 
#     "Best Speed":best_speed, 
#     "Sakthi Speed":speed_prediction,
#     "Match":match
# }
validate = {"angle":angle_prediction , "speed":speed_prediction}


df = pd.DataFrame(validate)
df.index += 1 
df.to_csv('Submission.csv', index_label="image_id")

32/32 [==============================] - 3s 76ms/step


In [6]:
speed_model = load_model("sakthi-speed2.hdf5")

test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)

93/93 [==============================] - 21s 168ms/step - loss: 0.0364 - accuracy: 0.9939 - mse: 0.0051 - auc_2: 0.9963
